In [1]:
import meshplot as mp
import ipywidgets
import numpy as np
import physics.kernels.vectormath as vm
import igl
import physics.physics as phys
import math
import time

In [2]:
import onshape.brepio as brepio

In [3]:
def add_axis(plot, center, x_dir, y_dir, z_dir, scale=1):
    V = np.array([center, center+x_dir * scale, center+y_dir * scale, center+z_dir * scale])
    Ex = np.array([[0,1]])
    Ey = np.array([[0,2]])
    Ez = np.array([[0,3]])
    plot.add_edges(V, Ex, shading={'line_color':'red'})
    plot.add_edges(V, Ey, shading={'line_color':'green'})
    plot.add_edges(V, Ez, shading={'line_color':'blue'})

In [4]:
loader = brepio.Loader('/projects/grail/benjones/cadlab')
did = '03180acbded3ea455c3e63ba'
mv = 'a5cc038a1c26099430d7314c'
eid = 'ee7cd5d7207130248b7845a2'

parts, mates = loader.load(did, mv, eid)
bounding_boxes = [parts[i][1].bounding_box() for i in parts]
maxdim = np.max([np.max(bbox[1,:]-bbox[0,:]) for bbox in bounding_boxes])

try:
    plot
except NameError:
    pass
else:
    del plot

for i in parts:
    g = parts[i]
    V = g[1].V.copy()
    F = g[1].F
    
    V = (g[0][:3,:3] @ V.T).T + g[0][:3,3]
    
    
    colors = None
    try:
        plot
    except NameError:
        plot = mp.plot(V, F, c=colors, return_plot=True)
    else:
        plot.add_mesh(V, F, c=colors)
        
for mate in mates:
    for mated in mate.matedEntities:
        tf = parts[mated[0]][0]
        newaxes = tf[:3, :3] @ mated[1][1]
        neworigin = tf[:3,:3] @ mated[1][0] + tf[:3,3]
        add_axis(plot, neworigin, newaxes[:,0], newaxes[:,1], newaxes[:,2], scale=maxdim/10)
plot

In [5]:
def mate_meshes(mate):
    meshes = []
    for mated in mate.matedEntities:
        tf = parts[mated[0]][0]
        V = parts[mated[0]][1].V.copy()
        F = parts[mated[0]][1].F
        V = np.ascontiguousarray((tf[:3,:3] @ V.T).T + tf[:3,3])
        #mp.plot(V, F)
        meshes.append((V, F))
    return meshes

def mate_phys_objs(mate, griddim, radius=-1):
    print('reinitializing physics objects')
    meshes = mate_meshes(mate)
    mated = mate.matedEntities[0]
    tf = parts[mated[0]][0]
    origin = tf[:3,:3] @ mated[1][0] + tf[:3,3]
    bounds = None
    if radius > 0:
        bounds = (origin - radius, origin + radius)
        print('bounds:',bounds,bounds[0].dtype)
    physObjs = [phys.PhysObject(V, F, griddim, bounds=bounds) for V, F in meshes]
    print('physobj rotation',physObjs[0].rotation)
    return (physObjs, bounds)

In [6]:
def joinmeshes(v1, f1, v2, f2):
    return np.vstack((v1, v2)), np.vstack((f1, f2 + v1.shape[0]))



def display_animation(v1, f1, v2, f2, states):
    fullv, fullf = joinmeshes(v1, f1, v2, f2)
    p = mp.plot(fullv, fullf) #TODO: render initial state
    n = len(states)
    play = ipywidgets.Play(min=0, max=n-1, step=1, interval=10, disabled=False)
    def ff(t):
        vs = []
        for j in range(2):
            v = (v1, v2)[j]
            R = vm.quaternion_to_matrix(states[t][1][j][0])
            x = states[t][1][j][1]
            newv = (R @ v.T).T + x
            vs.append(newv)
        fullv = np.vstack(vs)
        p.update_object(vertices=fullv)
    ipywidgets.interactive(ff, t=play)
    slider = ipywidgets.IntSlider(min=0, max=n-1, step=1)
    ipywidgets.jslink((play,'value'),(slider,'value'))
    return ipywidgets.HBox([play, slider])

In [9]:
griddim = maxdim/200
radius = 0.005

#simulators = []
for i,mate in enumerate(mates):
    print('mate',i)
    meshes = mate_meshes(mate)
        
    if len(meshes) == 2:
        start = time.time()
        physObjects = mate_phys_objs(mate, griddim, radius=radius)
        for obj in physObjects[0]:
            print('state:',obj.rotation, obj.translation)
        #physObjects = mate_phys_objs(mate, griddim)
        end = time.time()
        print('physObjects created in',end-start,'s')
        if physObjects[0][0].points.shape[0] == 0 or physObjects[0][1].points.shape[0] == 0:
            print('no points!')
            continue
        #p = mp.plot(physObjects[0][0].points + physObjects[0][0].translation[np.newaxis], shading={'point_size': 0.002})
        #p.add_points(physObjects[0][1].points + physObjects[0][1].translation[np.newaxis])
        
        #simulation bounds
        bounds = physObjects[1][0] - radius*.5, physObjects[1][1] + radius*.5
        
        sim = phys.Simulator(physObjects[0][0], physObjects[0][1], bounds[0], bounds[1], griddim, 100, 0)
        sim.integrate(0.001, 10)
        for state in sim.allstates:
            print(state)
        wid = display_animation(physObjects[0][0].V, physObjects[0][0].F, physObjects[0][1].V, physObjects[0][1].F, sim.allstates)
        display(wid)
        #simulators.append(sim)

mate 0
reinitializing physics objects
bounds: (array([-0.02125, -0.005  , -0.0055 ]), array([-0.01125,  0.005  ,  0.0045 ])) float64
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
physobj rotation [1. 0. 0. 0.]
state: [1. 0. 0. 0.] [0.00831881 0.00012332 0.00352896]
state: [1. 0. 0. 0.] [-1.6176173e-02  8.9838715e-05  7.3518585e-03]
physObjects created in 0.7077815532684326 s
copied data to GPU in 0.05276322364807129
(0.0, [(array([1., 0., 0., 0.], dtype=float32), array([0.00831881, 0.00012332, 0.00352896], dtype=float32)), (array([1., 0., 0., 0.], dtype=float32), array([-1.6176173e-02,  8.9838715e-05,  7.3518585e-03], dtype=float32))])
(0.001, [(array([ 0.11753564,  0.96533686, -0.01144272, -0.23276408], dtype=float32), array([0.1409571 , 0.12659265, 0.00352896], dtype=float32)), (array([ 8.0136734e-04, -3.4490782e-01,  3.4933624e-01,  8.7120730e-01],
      dtype=float32), array([-3.2552266 , -3.088313  ,  0.00735186], dtype=float32))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000193…

mate 1
reinitializing physics objects
bounds: (array([-0.02125, -0.005  ,  0.00275]), array([-0.01125,  0.005  ,  0.01275])) float64
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
physobj rotation [1. 0. 0. 0.]
state: [1. 0. 0. 0.] [-0.0324375   0.0002355   0.01176436]
state: [1. 0. 0. 0.] [-1.6176173e-02  8.9838715e-05  7.3518585e-03]
physObjects created in 0.41173481941223145 s
copied data to GPU in 0.02602863311767578
(0.0, [(array([1., 0., 0., 0.], dtype=float32), array([-0.0324375 ,  0.0002355 ,  0.01176436], dtype=float32)), (array([1., 0., 0., 0.], dtype=float32), array([-1.6176173e-02,  8.9838715e-05,  7.3518585e-03], dtype=float32))])
(0.001, [(array([0.05645216, 0.948504  , 0.17976952, 0.25462958], dtype=float32), array([-0.18405928,  0.23490568, -0.13490349], dtype=float32)), (array([ 0.00294308,  0.83549565,  0.40993968, -0.36590692], dtype=float32), array([ 2.3858433, -3.730966 ,  2.3438232], dtype=float32))])
(0.002, [(ar

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-6.247311…

mate 2
reinitializing physics objects
bounds: (array([ 0.019375, -0.005   , -0.0355  ]), array([ 0.029375,  0.005   , -0.0255  ])) float64
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
physobj rotation [1. 0. 0. 0.]
state: [1. 0. 0. 0.] [2.4388213e-02 5.2141175e-05 9.3903486e-03]
state: [1. 0. 0. 0.] [0.00831881 0.00012332 0.00352896]
physObjects created in 0.6404280662536621 s
no points!
mate 3
reinitializing physics objects
bounds: (array([ 0.019375, -0.005   ,  0.00663 ]), array([0.029375, 0.005   , 0.01663 ])) float64
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
physobj rotation [1. 0. 0. 0.]
state: [1. 0. 0. 0.] [ 2.4367658e-02 -5.6298457e-05  9.6064210e-03]
state: [1. 0. 0. 0.] [2.4388213e-02 5.2141175e-05 9.3903486e-03]
physObjects created in 0.2624807357788086 s
copied data to GPU in 0.0064258575439453125
(0.0, [(array([1., 0., 0., 0.], dtype=float32), array([ 2.4367658e-02,

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(7.3416158…

mate 4
reinitializing physics objects
bounds: (array([ 0.019375, -0.005   , -0.0097  ]), array([0.029375, 0.005   , 0.0003  ])) float64
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
rotation [1. 0. 0. 0.]
rotationEND [1. 0. 0. 0.]
physobj rotation [1. 0. 0. 0.]
state: [1. 0. 0. 0.] [ 2.4377003e-02 -3.5794761e-05 -2.7197287e-03]
state: [1. 0. 0. 0.] [2.4388213e-02 5.2141175e-05 9.3903486e-03]
physObjects created in 0.19522762298583984 s
copied data to GPU in 0.01797771453857422
(0.0, [(array([1., 0., 0., 0.], dtype=float32), array([ 2.4377003e-02, -3.5794761e-05, -2.7197287e-03], dtype=float32)), (array([1., 0., 0., 0.], dtype=float32), array([2.4388213e-02, 5.2141175e-05, 9.3903486e-03], dtype=float32))])
(0.001, [(array([ 0.00661147, -0.2400118 ,  0.80569524,  0.5414849 ], dtype=float32), array([-4.2466116, -4.1309404,  2.0201132], dtype=float32)), (array([ 0.0010989 ,  0.00327127, -0.00370438,  0.9999872 ], dtype=float32), array([ 0.36250576,  0.05161132, -0.11790521], dtype=float

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.003274…

# ### off-center collision
v, f = igl.read_triangle_mesh('data/cylinder3.obj')
grid_len = 0.02
density = 1
obj1 = phys.PhysObject(v, f, grid_len, translation=np.array([0, 0, 0], np.float32), density=density)
obj2 = phys.PhysObject(v, f, grid_len, translation=np.array([0.5, 2.1, 0], np.float32),
                    rotation=np.array([math.sqrt(2)/2, 0, math.sqrt(2)/2, 0], np.float32), density=density,
                    velocity=np.array([0, -5, 0], np.float32), angular_velocity=np.array([10, 0, 0], np.float32))

sim = phys.Simulator(obj1, obj2, np.array([-1.1, -1.1, -1.1]), np.array([1.1, 3.1, 1.1]), 0.02, 100, 0)
sim.integrate(0.001, 1000)
#print(sim.allstates)

display_animation(v, f, v, f, sim.allstates)